In [1]:
import mezze
import numpy as np
import pickle
import sys

In [ ]:
def get_free_evol_trajectories(config_specs, noise_specs):
    '''
    Creates free evolution trajectories based on the simulations configuration specifications and noise specifications
    
    noise_specs: assumes dephasing noise only, further flexibility can be obtained by changing the pmd
    '''
    print('Generating noisy qubit trajectories...')
    noise_amp = noise_specs['amp']
    corr_time = noise_specs['corr_time'] # This does nothing for this white noise model (it is effectively set to 0)
    noise_type = noise_specs['type']
    
    # Set the simulation parameters
    config = mezze.SimulationConfig() # Get the config structure (class)
    mezze.Noise.seed(1000) # Set the rng seed (doesn't work if parallel is set to true below)
    config.num_steps = config_specs['steps'] # Number of time steps to use in the simulation (larger numbers are more accurate)
    config.time_length = config_specs['T'] # Set the time length of the integration
    config.dt = float(config.time_length) / float(config.num_steps) # Set the time-step size
    config.parallel = False # Set to true for parallel Monte Carlo, False otherwise
    config.num_runs = 1 # This sets the number of Monte Carlo trials to use to compute the average superoperator
    config.sample_time(1) # This flag tells the code to report out the superoperator at every time step.
    config.get_unitary = True # Report unitary as opposed to Liouvillian
    time_axis=np.linspace(0., config.time_length, config.num_steps)

    # Instantiate the pmd object
    pmd = mezze.implementations.Z_noise_X_control_qubit()
    pmd.noise_hints[0]['corr_time'] = noise_amp
    pmd.noise_hints[0]['amp'] = corr_time
    pmd.noise_hints[0]['type'] = noise_type

    # Use the circuit class to generate primitive controls
    x_ctrl = np.zeros(config.num_steps)
    controls = [x_ctrl]
    
    u_targ = np.array([[1.0,0.0],[0.0,1.0]])
    realization_list = []
    for r in range(config_specs['num_runs']):
        # Create the Hamiltonian iterator
        ham_iterator = mezze.PrecomputedHamiltonian.create(pmd, controls, config)
        # Setup the Monte Carlo simulation
        mc = mezze.Simulation(config, pmd, ham_iterator)
        
        report = mc.run()
        realization_list.append(report)
    return realization_list

In [6]:
def compute_dd_propagator(sequence, u_list, dt):
    '''
    Computes dynamical decoupling propagator assuming instantaneous pulses
    
    sequence: specifications for the dynamical decoupling sequence
    u_list: list of free evolution trajectories generated by the mezze simulator
    dt: simulation time step size
    '''
    U_tot = np.eye(2)
    U_prev = U_tot
    Px = -1.0j*np.array([[0.0,1.0],[1.0,0.0]])
    Py = -1.0j*np.array([[0.0,-1.0j],[1.0j,0.0]])
    Pz = -1.0j*np.array([[1.0,0.0],[0.0,-1.0]])
    T = 0.0
    for elem in sequence:
        if type(elem) == str:
            if elem == 'X':
                U_tot = Px*U_tot
            elif elem == 'Y':
                U_tot = Py*U_tot
            elif elem == 'Z':
                U_tot = Pz*U_tot
        else:
            T += float(elem)
            n_val = int(T/dt)-1
            U_tot = (u_list.time_samples[n_val].liouvillian()*U_prev.conj().T)*U_tot
            U_prev = u_list.time_samples[n_val].liouvillian()
    return U_tot

# CPMG Example

In [14]:
def cpmg(cycles, time, pulse):
    '''
    Generates a CPMG sequence for Mezze simulation
    
    cycles: number of sequence repetitions
    time: total cycle time
    pulse: pulse operator specification, either 'X', 'Y', or 'Z'
    '''
    base = [0.25, pulse, 0.5, pulse, 0.25]
    seq = base
    for i in range(cycles-1):
        seq = base + seq
    seq = [float(elem)*(time/cycles) if elem != pulse else elem for elem in seq ]
    return seq

In [17]:
total_t = 1
dt = float(total_t/config_specs['steps'])
config_specs = {'steps': 1000, 'T': total_t, 'num_runs': 100}
noise_specs = {'amp': 0.001, 'corr_time': 5, 'type': 'gauss'}

u_realz = get_free_evol_trajectories(config_specs, noise_specs)

u_targ = np.array([[1.0,0.0],[0.0,1.0]])
F = 0.0
for uf in u_realz:
    seq = cpmg(cycles=3, time=total_t, pulse='X')
    utot = compute_dd_propagator(seq, uf, dt)
    F += 1.0/4.0*(1.0/config_specs['num_runs'])*abs(np.trace(np.dot(u_targ,utot)))**2.0
print('Average Gate Fidelity: %2.5f' % F)

Average Gate Fidelity: 0.54396
